In [2]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally

pybullet build time: Nov 28 2023 23:45:17


In [3]:
planeId = p.loadURDF("plane.urdf")

In [4]:
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robotId = p.loadURDF("../ik/robot.urdf", startPos, startOrientation)

In [5]:
import threading
def sim_main():
    while True:
        p.stepSimulation()
        time.sleep(1./240.)
sim_t = threading.Thread(target=sim_main)
sim_t.start()

↑ ここまでpybulletの初期化 (1回しか実行しちゃいけない類のやつ)

----

In [6]:
import webcface
wcli = webcface.Client("bullet-sim")
wcli.start()

In [7]:
import math
def rad2servo(angle: float) -> float:
    return (math.degrees(angle) + 150) * 1023 / 300
def servo2rad(angle: float) -> float:
    return math.radians(angle / 1023 * 300 - 150)

In [39]:
# dynamixelに仕様をあわせる
@wcli.func(args = [webcface.Arg(type=float)] * 7)
def move_servo(*angles):
    print(f"move_servo {angles}")
    arm_joint = [30, 31, 33, 35, 36, 37, 39]
    for i in range(7):
        p.setJointMotorControl2(
            robotId,
            arm_joint[i],
            p.POSITION_CONTROL,
            servo2rad(angles[i]),
            force=3,
            positionGain=0.1,
            velocityGain=1,
        )


move_servo (500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0)
move_servo (500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0)
move_servo (500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0)
move_servo (491.306876, 585.804882, 445.370512, 475.901144, 581.88576, 479.315718, 500.0)
move_servo (502.918516, 588.193221, 495.71787, 451.969544, 572.076862, 480.11123, 500.0)
move_servo (519.219167, 550.265088, 559.470088, 401.533316, 493.834071, 494.524433, 500.0)
move_servo (518.682004, 406.704099, 565.563476, 399.596883, 476.905504, 494.634566, 500.0)
move_servo (519.381039, 95.438109, 482.942641, 172.119657, 417.311614, 503.336215, 500.0)
move_servo (541.290816, 80.553702, 444.903857, 191.781521, 417.302717, 503.681849, 500.0)
move_servo (608.192423, 54.117759, 353.537715, 253.535948, 483.561716, 417.539027, 500.0)
move_servo (733.645259, 156.831588, 210.606106, 206.334888, 362.981379, 657.89414, 500.0)
move_servo (758.257417, 173.968556, 172.662759, 217.631287, 279.347944, 672.570768, 500.0)
move_

In [13]:
enable_update_pos = True
def update_pos():
    arm_joint = [30, 31, 33, 35, 36, 37, 39]
    while enable_update_pos:
        for i in range(1, 8):
            wcli.value(f"servo/{i}").set(rad2servo(p.getJointState(robotId, arm_joint[i - 1])[0]))
        wcli.sync()
        time.sleep(0.1)
threading.Thread(target=update_pos, daemon=True).start()

In [12]:
# enable_update_pos = False

In [10]:
p.setGravity(0,0,-10)

In [11]:
@wcli.func()
def reset_pos():
    p.resetBasePositionAndOrientation(robotId, startPos, startOrientation)
    p.resetBaseVelocity(robotId, [0, 0, 0], [0, 0, 0])

----

In [23]:
arm_joint = [30, 31, 33, 35, 36, 37, 39]
for i in range(1, 8):
    wcli.value(f"servo/{i}").set(rad2servo(p.getJointState(robotId, arm_joint[i - 1])[0]))
wcli.sync()
time.sleep(0.1)

In [38]:
wcli.value(f"servo/{i}").set(0)
wcli._send_thread

<Thread(Thread-7 (msg_send), started daemon 128516731434560)>

In [68]:
p.resetBasePositionAndOrientation(robotId, startPos, startOrientation)
p.resetBaseVelocity(robotId, [0, 0, 0], [0, 0, 0])

In [10]:
p.getJointState(robotId, 30)

(-0.3459644951688137,
 0.0006254333756990282,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 0.0013008215625082875)

In [5]:
p.getNumJoints(robotId)

47

In [6]:
for i in range(p.getNumJoints(robotId)):
    print(i, p.getJointInfo(robotId, i))

0 (0, b'base_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'base_link', (0.0, 0.0, 0.0), (0.0, 0.0, 0.0102), (0.0, 0.0, 0.0, 1.0), -1)
1 (1, b'wheel_left_joint', 0, 7, 6, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'wheel_left_link', (0.0, 0.0, 1.0), (-0.01, 0.115, 0.025), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
2 (2, b'wheel_right_joint', 0, 8, 7, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'wheel_right_link', (0.0, 0.0, 1.0), (-0.01, -0.115, 0.025), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
3 (3, b'caster_front_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'caster_front_link', (0.0, 0.0, 0.0), (0.10500000000000001, 0.0, 0.007), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
4 (4, b'caster_back_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'caster_back_link', (0.0, 0.0, 0.0), (-0.14500000000000002, 0.0, 0.009), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
5 (5, b'gyro_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'gyro_link',

In [7]:
arm_joint = [30, 31, 33, 35, 36, 37, 39]

In [16]:
p.setJointMotorControl2(
    robotId,
    arm_joint[0],
    controlMode=p.POSITION_CONTROL,
    targetPosition=0
)
p.setJointMotorControl2(
    robotId,
    arm_joint[1],
    controlMode=p.POSITION_CONTROL,
    targetPosition=1
)
p.setJointMotorControl2(
    robotId,
    arm_joint[2],
    controlMode=p.POSITION_CONTROL,
    targetPosition=1
)

In [67]:
for i in range(7):
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=0,
        force=10
    )

In [70]:
# 1個ずつ1rad→0radに動かす
for i in range(7):
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=1,
        force=1
    )
    time.sleep(2)
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=0,
        force=1
    )
    time.sleep(2)

KeyboardInterrupt: 